Using Pandas and Numpy with Data in HTML Tables
--

First, we'll need to import the libraries.

In [ ]:
import pandas as pd
import numpy as np

This is an example of how to read a table from Wikipedia, harvest some data, and change it around so that it reads well for NumPy and Pandas.

This first entry describes the `.read_html()` function from the Pandas library. If you'd like to see the help, uncomment (delete the `#`) and re-run the line. If you do, remember to put the comment back in when you're done. Otherwise, you'll see the help box any time you run the page.

In [ ]:
#pd.read_html?

In [ ]:
season_URL = r'https://en.wikipedia.org/wiki/Parks_and_Recreation_%28season_1%29'
season_page = pd.read_html(season_URL,header=0)
season_page

Above, we've passed two arguments to the pd.read_html() method. First, we're passing the URL. Pandas will use other installed libraries to connect to this URL and begin extracting the tables as a list. We're also specifying that the first row (0) is where the header information is.

In the resulting text, notice the '[' at the beginning, and the ']' all the way at the end? That should be an indication that it is a list.  Of course you could always:

In [ ]:
type(season_page)

Since it is a list, it can be sliced. We just need to figure out which slice we want. Is it the first one? (remember, Python is zero-indexed.)

In [ ]:
season_page[0]

While we could do some things with this, this table isn't the full data we're looking to process. For this page, we're looking at the second table, or `season_page[1]`. We'll go ahead and assign that to a variable we're calling 'season'.

In [ ]:
season_data = season_page[1]
season_data

Notice here that there are a bunch of cells with 'NaN' in them. We talked about 'Not a Number' before, and again NumPy is saying that it couldn't find or interpret data here. We could use some Pandas functions to deal with these cells.

In [ ]:
season_data.dropna()

The .dropna() method effectively drops any rows where there are one or more NaN entries. Unfortunately, while there were rows that contained NaN, there were still some cells in those rows with data in them. By default, .dropna() has the argument **how='any'**. Meaning, if a row contains one or more NaN, then .dropna() will drop it. Instead, we can use `.dropna(how='all')`. This will only drop rows where all cells are NaN.

In [ ]:
season_data.dropna(how='all')

This doesn't show anything here, as we don't have any rows that are completely filled with NaN. But, we can use .fillna() to automatically change that value to something else that may be relevant to your data collection methods.

In [ ]:
season_data.fillna(-999)

.fillna() has multiple methods, like forward fill, backward fill, interpolation.

In [ ]:
#season_data.fillna?

In [ ]:
season_data.fillna(method='ffill')

Check out what `.fillna('ffill')` did! Anywhere there is a `NaN` it copied the data from the cell previous. This is close to what we want...

In [ ]:
season_data.fillna(method='backfill')

ok! Now we've taken the season description from the following rows and _backfilled_ it into the `NaN` cells. If we're ok with that, we can reassign the new info into the previous variable. Sometimes, you can add the keyword argument, _kwarg_, `inplace=True` to make the change in place. In other words, it saves us an assignment statement.

In [ ]:
season_data.fillna(method='backfill',inplace=True)

While we're at it, let's rename that column called `Unnamed: 7` to something more informative, like `Description`. Here we're using the DataFrame method `.rename()` with the kwarg `columns=` and a _dictionary_ to give a one-to-one relationship between the items we want to rename, and how we want to _rename_ them. Although below we're only renaming one, we can definitely do more than that with multiple key-value pairs.

In [ ]:
season_data.rename(columns={'Unnamed: 7':'Description'},inplace=True)

In [ ]:
season_data.head()

Notice here that we do have data in each of the cells, but the descriptions are duplicated. We've already gotten the data we need out of the cells and backfilled it into the place we need it. Let's get rid of the lines we don't need. How do we do that?

hint: see how _every other_ line is a row of descriptions we don't need? I wonder where we've seen that before...

Check this out:

In [ ]:
season_data.iloc[0::2]

That's just like our slicers! **AND** we've put it in an indexer. 

Before, we used `.loc[]` to work with grades. Let's take a look at our indexer options again. `.loc[]` and `.iloc[]`. For the most part, if you're going to use labeled indicators, you'll use `.loc[]`. If you're going to use integer positions (remember the _zero-indexed_) you'll use `.iloc[]`.

In [ ]:
season_info = season_data.iloc[0::2]

Setting the index to one of the unique Series in the DataFrame. 'No. Overall' makes sense.

In [ ]:
season_info.set_index('No.overall',inplace=True)

In [ ]:
season_info

For the sake of space, the preview here doesn't show all the information, but you can definitely check the contents by using the `.values` property.

In [ ]:
season_info['Description'].values

You can also set the width of the columns to allow visibility of all of the data like this:

In [ ]:
pd.set_option('display.max_colwidth',None)

In [ ]:
season_info

By default, the maximum column width is 50, so we'll go ahead and set that back here:

In [ ]:
pd.set_option('display.max_colwidth',50)

Back to indexers... Indexers can be used like slices. They have the brackets and colons, but they can also be used in conjunction with commas to indicate two dimensions. In the first example, we use our knowledge of slices on a DataFrame to separate content out in two different ways. We're also using the NumPy `.all()` method to verify that the two sections are equivalent. We can also check the number of entries by using the `.len()` method.

In [ ]:
print(np.all(season_data[1::2].iloc[:,0] == season_data.iloc[1::2,0]))
print(len(season_data[1::2].iloc[:,0]))
print(len(season_data.iloc[1::2,0]))

Be careful when mixing integers with strings for `.loc[]` and `.iloc[]`. If you need non-contiguous ranges, you can use tuples. We haven't talked much about tuples, but they're basically lists that are _immutable_ to a degree. We won't go too far down this road. Basically, if it looks like a list, but it's surrounded by parentheses, it's a tuple.

In [ ]:
print(type(season_info))
print(type(season_info.iloc[2:4,1:5]))
print(type(season_info.iloc[2:5].loc[:,('Title','Written by')]))

In [ ]:
season_info.iloc[2:4,1:5]
season_info.iloc[2:5].loc[:,('Title','Written by')]

In fact, all of these are functionally equivalent:

In [ ]:
# .loc uses strings to locate cells
season_info.loc[:,'Title']

In [ ]:
# .iloc uses integers to locate cells
season_info.iloc[:,1] 

In [ ]:
# alternatively, you can refer to the specific column head
season_info['Title']

In [ ]:
# or use dot notation (if there are no spaces in the column head).
season_info.Title

In [ ]:
np.all(season_info.loc[:,'Title'] == season_info.iloc[:,1])

In [ ]:
season_info

Looking at the viewer numbers, they are formatted like this:

`4.70[19]`

Unfortunately, Pandas doesn't know what to do with this; the format isn't anything it recognizes. So, we're going to temporarily convert it to a string (using the `.str` accessor), and use some string methods on it. Specifically, we'll use the `.split()` method again. This time, however, there isn't a space to _split_ on. As we mentioned before, the `.split()` method allows you to split on a specified character - we'll use the open bracket, as it separates the number from the footnote.

In [ ]:
season_info.iloc[:,5].str.split('[')

See how there is one left-bracket, and two right-brackets on each line? Each cell is divided into a two-item list: The numbers before the '[', and everything else. It looks a little confusing, I'll admit.

We only care about the stuff in the front, so we'll grab it with (you guessed it) a slice. While we're at it, we're going to go ahead and convert everything to a float, so we can have a number with a decimal.

In [ ]:
season_info.iloc[:,5] = season_info.iloc[:,5].str.split('[').str[0].astype('float')

And, since it is a number (specifically, a float) we can do math operations on it.

In [ ]:
season_info.iloc[:,5].median()

Now, we're going to use the .apply() method to apply a function to a column. We're going to convert the 'Original air date' column to a DateTime object. Pandas will try to see if there are ways to interpret the information in these cells as dates. First, let's look at the `pd.to_datetime` method.

In [ ]:
#pd.to_datetime?

In [ ]:
season_info['Original air date'].apply(pd.to_datetime)

This information, `May 14, 2009`, thankfully, is in a format that Pandas can convert with `pd.to_datetime`.

*we'll be coming back here later...*

In [ ]:
season_info['Original air date'] = season_info['Original air date'].apply(pd.to_datetime)

Now, let's do some graphing!

First, we'll tell jupyter that we want to do the graphs inline, and then we'll import another library.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

Graphing can be as simple as using the `.plot()` method.

In [ ]:
season_info.iloc[:,5:7].plot(); 

But what if we want to get really fancy? PyPlot has some really excellent features. We can append this stuff to the default figure 1.

In [ ]:
#plt.plot?

In [ ]:
print(type(season_info.iloc[:,1]))
print(type(season_info.iloc[:,1].values))

In [ ]:
#plt.xticks?

In [ ]:
episode_titles = season_info.iloc[:,1].values
episode_viewers = season_info.iloc[:,5].values

index = np.arange(len(season_info))

plt.figure(num=None, figsize=(16, 8), facecolor='w', edgecolor='k')
plt.xlabel('Episode Titles')
plt.title('Parks & Recreation Viewer Stats')
plt.xticks(index,episode_titles,rotation=90)
plt.ylabel('Viewers (in millions)')
plt.plot(index, episode_viewers,'b',label='Viewers')
plt.plot(pd.Series(episode_viewers).rolling(window=3).mean(),'r',label='Rolling Mean')
plt.grid()

plt.legend()
plt.show()